<a href="https://colab.research.google.com/github/pchelkas/movie_reviews/blob/master/movie_reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd


df = pd.read_csv('output.csv')


In [0]:
from sklearn.model_selection import train_test_split
for row in df:
    sentiment = 2

    if row['target'] == 'bad':
        sentiment = -1
    elif row['target'] == 'neutral':
        sentiment = 0
    else:
        sentiment = 1

    if sentiment == 2:
        continue

    x.append(row['review_text'])
    y.append(sentiment)
        
        
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

TypeError: string indices must be integers

In [0]:
data_positive = df[df['target'] == 'good']['review_text']
data_neutral = df[df['target'] == 'neutral']['review_text']
data_negative = df[df['target'] == 'bad']['review_text']

In [0]:
sample_size = min(data_positive.shape[0], data_negative.shape[0], data_neutral.shape[0])

raw_data = np.concatenate((data_positive.values[:sample_size],
                           data_negative.values[:sample_size], data_neutral.values[:sample_size]), axis=0)

In [0]:
labels = [1] * sample_size + [-1] * sample_size + [0] * sample_size

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(raw_data, labels, test_size=0.2, random_state=42)

In [0]:
from keras.preprocessing import text, sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

max_length = 5000
max_features = 20000
embedding_dim = 300

x_all = []
x_all.extend(X_test)
x_all.extend(X_train)

tk = Tokenizer(num_words=max_features, lower=True, filters='\n\t')
tk.fit_on_texts(x_all)
x_train_seq = tk.texts_to_sequences(X_train)
x_test_seq = tk.texts_to_sequences(X_test)

np_x_train = pad_sequences(x_train_seq, maxlen=max_length,  padding='post')
np_x_test = pad_sequences(x_test_seq, maxlen=max_length,  padding='post')
np_y_train = to_categorical(y_train)

class_num = np_y_train.shape[1]

print ('np_x_train shape: {}'.format(np_x_train.shape))
print ('np_x_test shape: {}'.format(np_x_test.shape))
print ('np_y_train shape: {}'.format(np_y_train.shape))

np_x_train shape: (17474, 5000)
np_x_test shape: (4369, 5000)
np_y_train shape: (17474, 2)


In [0]:
from keras.models import Model
from keras.layers import *
from keras.callbacks import EarlyStopping

def one_input_classifier(max_length, max_features, embedding_dim, class_num):
    inputs = Input(shape=(max_length,), name='input_1')
    embeddings = Embedding(max_features, embedding_dim, input_length=max_length, name='embedding_1')(inputs)

    conv_1 = Conv1D(32, 9, activation='relu', name='conv1d_1')(embeddings)
    maxpool_1 = MaxPooling1D(16, name='maxpool1d_1')(conv_1)
    dropout_1 = Dropout(0.2, name='dropout_1')(maxpool_1)

    conv_2 = Conv1D(32, 7, activation='relu', name='conv1d_2')(dropout_1)
    maxpool_2 = MaxPooling1D(8, name='maxpool1d_2')(conv_2)
    dropout_2 = Dropout(0.2, name='dropout_2')(maxpool_2)

    bilstm = Bidirectional(LSTM(32, dropout=0.2, recurrent_dropout=0.2, name='lstm_1'),
        name='bidirectional_1')(dropout_2)
    preds = Dense(class_num, activation='softmax', name='preds')(bilstm)

    model = Model(inputs=inputs, outputs=preds)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=1,
                               mode='min', baseline=None, restore_best_weights=True)

models = []
classifier_num = 10

for i in range(classifier_num):
    model = one_input_classifier(max_length, max_features, embedding_dim, class_num)

    if i == 0:
        print(model.summary())

    model.fit(np_x_train, np_y_train, validation_split=0.3, shuffle=True,
              callbacks=[early_stopping], epochs=10, batch_size=32, verbose=1)
    models.append(model)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 5000)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 5000, 300)         6000000   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 4992, 32)          86432     
_________________________________________________________________
maxpool1d_1 (MaxPooling1D)   (None, 312, 32)           0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 312, 32)           0         
_________________________________________________________________
conv1d_2 (Conv1D)    